In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

[![View on GitHub][github-badge]][github-keras-v2] [![Open In Colab][colab-badge]][colab-keras-v2] [![Open in Binder][binder-badge]][binder-keras-v2]

[github-badge]: https://img.shields.io/badge/View-on%20GitHub-blue?logo=GitHub
[colab-badge]: https://colab.research.google.com/assets/colab-badge.svg
[binder-badge]: https://static.mybinder.org/badge_logo.svg

[github-keras-v2]: https://github.com/mbrukman/reimplementing-ml-papers/blob/main/lenet/keras/LeNet_Keras_v2_custom_Subsampling_layer_and_activation.ipynb
[colab-keras-v2]: https://colab.research.google.com/github/mbrukman/reimplementing-ml-papers/blob/main/lenet/keras/LeNet_Keras_v2_custom_Subsampling_layer_and_activation.ipynb
[binder-keras-v2]: https://mybinder.org/v2/gh/mbrukman/reimplementing-ml-papers/main?filepath=lenet/keras/LeNet_Keras_v2_custom_Subsampling_layer_and_activation.ipynb

In [ ]:
%%bash

readonly GH_USER="mbrukman"
readonly GH_REPO="reimplementing-ml-papers"
readonly GH_BRANCH="main"

# Download the library for processing MNIST dataset as well as our custom
# Subsampling layer, activation function, and LeNet model.
for path in datasets/mnist/mnist_keras.py \
            lenet/keras/{activations,lenet,subsampling}.py ; do
  module="$(basename "${path}")"
  if ! [ -f "${module}" ]; then
    curl -s -o "${module}" "https://raw.githubusercontent.com/${GH_USER}/${GH_REPO}/${GH_BRANCH}/${path}"
  fi
done

In [ ]:
from tensorflow import keras

# Local imports downloaded above.
from activations import scaled_tanh
from lenet import LeNet
from subsampling import Subsampling

from mnist_keras import MNIST

In [ ]:
# Define the model architecture.
model = LeNet(subsampling=Subsampling, activation=scaled_tanh)

model.summary()

Model: "LeNet-5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 C1 (Conv2D)                 (None, 28, 28, 6)         156       
                                                                 
 S2 (Subsampling)            (None, 14, 14, 6)         12        
                                                                 
 S2_act (Activation)         (None, 14, 14, 6)         0         
                                                                 
 C3 (Conv2D)                 (None, 10, 10, 16)        2416      
                                                                 
 S4 (Subsampling)            (None, 5, 5, 16)          32        
                                                                 
 S4_act (Activation)         (None, 5, 5, 16)          0         
                                                                 
 C5 (Conv2D)                 (None, 1, 1, 120)         4812

In [ ]:
# Compile the model with optimizer and loss function.
opt = keras.optimizers.Adam(learning_rate=0.001)
loss_fn = keras.losses.CategoricalCrossentropy()
model.compile(optimizer=opt, loss=loss_fn, metrics=['accuracy'])

For details on the MNIST dataset including a data exploration, see [MNIST directory in my repo](https://github.com/mbrukman/reimplementing-ml-papers/tree/main/datasets/mnist).

Here, we will import a shared library to process the MNIST dataset into the format that we need to use below for model training and testing.

In [ ]:
%%capture --no-stderr

# This will download the MNIST dataset via the Keras library which outputs data
# to stdout, so we silence it above to avoid extraneous output.
mnist_data = MNIST()

In [ ]:
# Train the model.
#
# In this notebook, we scale the input into the range [0.0, 1.0] and convert the
# labels y to a categorical (one-hot) encoding from the default numeric values.
#
# For consistency, we use the same transformations for the test dataset below.
model.fit(mnist_data.x_train_scale_0_1(),
          mnist_data.y_train_categorical(),
          epochs=20)

Epoch 1/20
1875/1875 [==============================] - 16s 4ms/step - loss: 0.2747 - accuracy: 0.9146
Epoch 2/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0920 - accuracy: 0.9722
Epoch 3/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0606 - accuracy: 0.9815
Epoch 4/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0451 - accuracy: 0.9860
Epoch 5/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0352 - accuracy: 0.9890
Epoch 6/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0309 - accuracy: 0.9903
Epoch 7/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0245 - accuracy: 0.9917
Epoch 8/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0196 - accuracy: 0.9934
Epoch 9/20
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0193 - accuracy: 0.9932
Epoch 10/20
1875/1875 [==============================] - 7s 4ms/step - loss: 0.01

In [ ]:
# Evaluate the model.
#
# Note that we use the same input range scaling and label encoding as above.
model.evaluate(mnist_data.x_test_scale_0_1(), mnist_data.y_test_categorical())

313/313 [==============================] - 1s 3ms/step - loss: 0.0683 - accuracy: 0.9853


[0.06825553625822067, 0.9853000044822693]